<a href="https://colab.research.google.com/github/anandmali/Movies_Recommenders/blob/main/retrieval_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Basic TODO to create a model

1.   Install / Import required models.
2.   Loading and preparing datasets from the tenserflow collection. (This can be diffeerent source depending on project data storage).
3.   Since this is two tower model, we can create query tower and candidate tower and then combine both for final model.
4.   Create task and loss functions.
5. Combine above to create full model class.

#### Imports

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 17.5 MB/s eta 0:00:00


In [3]:
import os
import pprint

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

####Retrieving and Preparing data

#### movielens/100k-ratings
This dataset contains 100,000 ratings from 943 users on 1,682 movies. This dataset is the oldest version of the MovieLens dataset.

In [4]:
# Getting ratings dataset.
ratings = tfds.load("movielens/100k-ratings", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/0.1.1.incompleteWYUK5X/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


#### movielens/100k-movies
This dataset contains data of 1,682 movies rated in the 100k dataset. Dict with id, title, and genres.

In [5]:
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete1EHGMD/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})

# Printing for evaluation
for x in ratings.take(10).as_numpy_iterator():
  pprint.pprint(x)

movies = movies.map(lambda x: x["movie_title"])

for x in movies.take(10).as_numpy_iterator():
  pprint.pprint(x)


{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}
{'movie_title': b'Strictly Ballroom (1992)', 'user_id': b'92'}
{'movie_title': b'Very Brady Sequel, A (1996)', 'user_id': b'301'}
{'movie_title': b'Pulp Fiction (1994)', 'user_id': b'60'}
{'movie_title': b'Scream 2 (1997)', 'user_id': b'197'}
{'movie_title': b'Crash (1996)', 'user_id': b'601'}
{'movie_title': b'Aladdin (1992)', 'user_id': b'710'}
{'movie_title': b'True Romance (1993)', 'user_id': b'833'}
{'movie_title': b'Bob Roberts (1992)', 'user_id': b'916'}
{'movie_title': b'Starship Troopers (1997)', 'user_id': b'940'}
b'You So Crazy (1994)'
b'Love Is All There Is (1996)'
b'Fly Away Home (1996)'
b'In the Line of Duty 2 (1987)'
b'Niagara, Niagara (1997)'
b"Young Poisoner's Handbook, The (1995)"
b'Age of Innocence, The (1993)'
b'Flirt (1995)'
b'Frisk (1995)'
b'unknown'


#### Split data for training and evaluation

To fit and evaluate the model, we need to split it into a training and evaluation set. In an industrial recommender system, this would most likely be done by time: the data up to time  T  would be used to predict interactions after  T .

In this simple example, however, let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

length = len(ratings)

#Defining percentage of split of data for training and testing.
train_take = int(0.75 * length)
test_take = int(0.25 * length)

train = shuffled.take(train_take)
test = shuffled.skip(train_take).take(test_take)

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

This is required to help in lookup of features associated with the indices in querry tower.

In [8]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

t = unique_movie_titles[:10]
pprint.pprint(t)

i = unique_user_ids[:10]
pprint.pprint(i)


array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)
array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)


#### Two tower model

define dimesion of both towers

In [9]:
embedding_dimension = 32

#### User embedding tower (Query tower)

In [12]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

#### Movies features tower (Candidate tower)

In [13]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

#### Metrics
In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

In [14]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

#### Loss
The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In [15]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

Full model

In [16]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

#### Fitting and evaluating

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime


In [20]:
#Initiate the model
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

#Shuffle
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Train
model.fit(cached_train, epochs=3)

#Evaluate
model.evaluate(cached_test, return_dict=True, callbacks=[tensorboard_callback])

Epoch 1/3
10/10 [==============================] - 34s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0033 - factorized_top_k/top_5_categorical_accuracy: 0.0205 - factorized_top_k/top_10_categorical_accuracy: 0.0418 - factorized_top_k/top_50_categorical_accuracy: 0.1820 - factorized_top_k/top_100_categorical_accuracy: 0.3113 - loss: 58191.6731 - regularization_loss: 0.0000e+00 - total_loss: 58191.6731
Epoch 2/3
10/10 [==============================] - 33s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0271 - factorized_top_k/top_10_categorical_accuracy: 0.0540 - factorized_top_k/top_50_categorical_accuracy: 0.2177 - factorized_top_k/top_100_categorical_accuracy: 0.3548 - loss: 57176.3406 - regularization_loss: 0.0000e+00 - total_loss: 57176.3406
Epoch 3/3
7/7 [==============================] - 12s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.2000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.00

{'factorized_top_k/top_1_categorical_accuracy': 0.0005200000014156103,
 'factorized_top_k/top_5_categorical_accuracy': 0.004279999993741512,
 'factorized_top_k/top_10_categorical_accuracy': 0.010999999940395355,
 'factorized_top_k/top_50_categorical_accuracy': 0.08587999641895294,
 'factorized_top_k/top_100_categorical_accuracy': 0.18731999397277832,
 'loss': 2422.21630859375,
 'regularization_loss': 0,
 'total_loss': 2422.21630859375}

In [ ]:
%tensorboard --logdir logs/fit

#### Recommending movie title

In [18]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get some recommendations for given user ID.
_, titles = index(np.array(["54"]))
print(f"Top 10 recommended movies for user 54: \n{titles[0, :10, ]}")


Top 10 recommended movies for user 54: 
[b'Fled (1996)' b'Daylight (1996)'
 b"Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)"
 b'Face/Off (1997)' b'Eraser (1996)' b'Broken Arrow (1996)'
 b'Con Air (1997)' b'Arrival, The (1996)' b'Trigger Effect, The (1996)'
 b'Fear (1996)']
